In [1]:
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate
# vLLM itself
!pip install vllm

# Hugging Face transformers for model handling
!pip install transformers

# BitsAndBytes (for quantization support like 4-bit/8-bit)
!pip install bitsandbytes

# Accelerate (required for model device mapping)
!pip install accelerate

# For caching models and logging
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.0/180.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 100.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 1.9 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 8.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 3.9 MB/s eta 0:00:00:00:0100:01

In [2]:
import torch

# Check GPU status
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print(f"Available Memory: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB")

# Clear cache
torch.cuda.empty_cache()

GPU: Tesla T4
Total Memory: 15.83 GB
Available Memory: 15.83 GB


In [3]:
import os
os.environ['VLLM_USE_V1'] = '0'  # Disable V1 engine

# Clear any existing GPU memory
import torch
import gc

# Clear CUDA cache
torch.cuda.empty_cache()
gc.collect()

# Try to free all CUDA memory
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        with torch.cuda.device(i):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            
import pandas as pd
from vllm import LLM, SamplingParams
from tqdm import tqdm
import gc
import torch

# Load the CSV
df = pd.read_csv('/kaggle/input/banglachq/train.csv')
print(f"Loaded {len(df)} rows from train.csv")
print(f"Columns: {df.columns.tolist()}")

# Load model with vLLM on single GPU with optimized settings
model_name = "Qwen/Qwen2.5-3B-Instruct"
print(f"\nLoading {model_name} with vLLM...")

llm = LLM(
    model=model_name,
    trust_remote_code=True,
    dtype="float16",
    gpu_memory_utilization=0.65,  # Reduced slightly for larger context
    max_model_len=2048,  # Increased to 2048
    max_num_seqs=24,  # Reduced batch size slightly
)

# Get tokenizer for chat template
tokenizer = llm.get_tokenizer()

print("Model loaded successfully!\n")

# Translation function (kept prompt intact)
def translate_to_sylheti(bangla_text):
    prompt = f"""You are a precise translation tool. Your task is to translate the given Bangla text to Sylheti dialect using Bengali script.

        INSTRUCTIONS:
        - Analyze the text step-by-step before translating
        - Use only Bengali script in your final translation
        - After your reasoning, provide ONLY the final translation with no additional commentary

        Here are examples showing the translation process:

        Example 1:
        Bangla: আপনি কি নিয়মিত স্ক্যান করেন?
        Reasoning: "আপনি" becomes "আফনে" (phonetic shift প→ফ, ই→ে), "কি নিয়মিত" becomes "কিতা রেগুলার" (কি→কিতা with emphasis particle, নিয়মিত→রেগুলার lexical change), "স্ক্যান করেন" becomes "স্ক্যান খরইননি?" (করেন→খরইননি with খ sound and -ননি ending for question)
        Sylheti: আফনে কিতা রেগুলার স্ক্যান খরইননি?

        Example 2:
        Bangla: কোথায় কোচিং করেন?
        Reasoning: "কোথায়" becomes "কোনানো" (locative form কোথায়→কোনানো with extended vowel), "কোচিং" stays as "কোচিং" (unchanged), "করেন" becomes "খরইন?" (verb করেন→খরইন with খ sound and -ইন ending)
        Sylheti: কোনানো কোচিং খরইন?

        Example 3:
        Bangla: ডাল সিদ্ধ হয়েছে?
        Reasoning: "ডাল" becomes "ডাইল" (vowel shift আ→আই for diphthong), "সিদ্ধ" stays as "সিদ্ধ" (unchanged), "হয়েছে" becomes "অইছেনি?" (হয়েছে→অইছেনি with অ prefix and -নি question marker)
        Sylheti: ডাইল সিদ্ধ অইছেনি?

        Example 4:
        Bangla: এই যে মিয়া, কি নিয়ে যাও?
        Reasoning: "এই যে মিয়া" becomes "ওবা" (colloquial address form, complete lexical replacement for calling attention), "কি নিয়ে" becomes "কিতা লইয়া" (কি→কিতা, নিয়ে→লইয়া with ল sound), "যাও" becomes "যাইরায়?" (যাও→যাইরায় with extended verb form and য় ending)
        Sylheti: ওবা, কিতা লইয়া যাইরায়?

        Example 5:
        Bangla: জুম্মার পর যাব।
        Reasoning: "জুম্মার" becomes "জুম্মার" (unchanged as Arabic loanword), "পর" stays as "পর" (unchanged), "যাব" becomes "যামু" (first person future -ব→-মু typical Sylheti verb ending)
        Sylheti: জুম্মার পর যামু।

        Now translate this text:
        Bangla: {bangla_text}

        Think through the translation:
        - Identify key vocabulary transformations
        - Note grammatical changes (verb forms, pronouns, question markers)
        - Apply Sylheti phonetic patterns (খ for ক in verbs, diphthongs, vowel shifts)

        Provide your final translation below:
        Sylheti:"""

    messages = [{"role": "user", "content": prompt}]

    # Apply chat template
    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    return formatted_prompt

# Set sampling parameters
sampling_params = SamplingParams(
    temperature=0.3,
    top_p=0.9,
    max_tokens=200
)

# Process in smaller batches to avoid OOM
BATCH_SIZE = 32  # Match max_num_seqs
all_translations = []

print(f"Processing {len(df)} sentences in batches of {BATCH_SIZE}...")

for batch_start in tqdm(range(0, len(df), BATCH_SIZE), desc="Processing batches"):
    batch_end = min(batch_start + BATCH_SIZE, len(df))
    batch_df = df.iloc[batch_start:batch_end]
    
    # Prepare prompts for this batch
    batch_prompts = []
    for idx, row in batch_df.iterrows():
        bangla_text = row['question']
        prompt = translate_to_sylheti(bangla_text)
        batch_prompts.append(prompt)
    
    # Generate translations for this batch
    outputs = llm.generate(batch_prompts, sampling_params)
    
    # Extract translations
    batch_translations = [output.outputs[0].text.strip() for output in outputs]
    all_translations.extend(batch_translations)
    
    # Clear memory after each batch
    del outputs, batch_prompts
    gc.collect()
    torch.cuda.empty_cache()

# Print first few examples
print("\nFirst 3 examples:")
for idx in range(min(3, len(df))):
    print(f"\nBangla: {df.iloc[idx]['question']}")
    print(f"Sylheti: {all_translations[idx]}")

# Add translations to dataframe
df['sylheti_translation'] = all_translations

# Save results
df.to_csv('qwen-5-shot-sylheti-CoT', index=False)
print(f"\n✅ Translation complete! Saved to 'qwen_10_train_with_sylheti_translations.csv'")
print(f"Total sentences translated: {len(df)}")





2025-09-29 13:31:52.452000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759152712.660044      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759152712.721619      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 09-29 13:32:07 [__init__.py:216] Automatically detected platform cuda.
Loaded 1880 rows from train.csv
Columns: ['id', 'question', 'indices', 'summary']

Loading Qwen/Qwen2.5-3B-Instruct with vLLM...
INFO 09-29 13:32:08 [utils.py:328] non-default args: {'trust_remote_code': True, 'dtype': 'float16', 'max_model_len': 2048, 'gpu_memory_utilization': 0.65, 'max_num_seqs': 24, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-3B-Instruct'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

INFO 09-29 13:32:23 [__init__.py:742] Resolved architecture: Qwen2ForCausalLM


...

Processing batches:  98%|█████████▊| 58/59 [36:08<00:37, 37.50s/it]

Adding requests:   0%|          | 0/24 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/24 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processing batches: 100%|██████████| 59/59 [36:34<00:00, 37.19s/it]


First 3 examples:

Bangla: আমার বয়স ২৫ বছর । আজ কিছুদিন যাবত আমার ঘন ঘন টয়লেট হচ্ছে । হটাৎ পেটের মধ্যে ব্যাথা অনুভূত হয়ে টয়লেটে চাপ দেয় এবং টয়লেটে যাওয়ার পরে ও টয়লেট ক্লিয়ারলি হচ্ছে না । এই সমস্যাটার সমাধান জানাবেন । আর একটি সমস্যা আছে , সেটি হল আজ দুই দিন যাবত হটাৎ করে আমার ঠান্ডা লেগে সর্দি + অনেক হাচি হচ্ছে । 
Sylheti: আফনে বয়স আই আট পাঁচ বছর । আজ কিতা দিন খরইননি আমার ঘন ঘন টয়লেট অইছে । হটাৎ পেটের মধ্যে ব্যাথা অনুভূত হয়ে টয়লেটে খরইননি চাপ দেয় এবং টয়লেটে খরইননি যাইরায় ও টয়লেট ক্লিয়ারলি অইছে না ।

Bangla: বাচ্চার বয়স ১০ মাস চলছে , ওজন ৮ . ৩ কেজি । দাঁত উঠতেছে , হালকা ঘন সর্দি আজ ৪ দিন যাবত , হাছি কম । মায়ের বুকের দুধ ছাড়া অন্য কোন খাবার ১০ দিন ধরে খায় না , এমন কি পানিও কম খায় । সর্দি ভাল করার জন্য উপায় কী এবং খাবার খাওয়ানোর ব্যাপারে কী করব ? 
Sylheti: বাচ্চার বয়স আইছেনি দুই মাস চলে, ওজন আইছেনি আট টেন কেজি । দাঁত উঠেনি, হালকা ঘন সর্দি আজ চার দিন যাবত, হাছি কম । মায়ের বুকের দুধ ছাড়া অন্য কোন খাবার দশ দিন ধরে খায়নি, এমন কি পানি কম খায়নি

Bangla: আমার সব সময় চুল পড়ে । প্রা